<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Evaluating and Improving Search and Retrieval Applications</h1>

Imagine you're an engineer at Arize AI and you've built and deployed a documentation question-answering service using LangChain and Qdrant. Users send questions about Arize's core product via a chat interface, and your service retrieves chunks of your indexed documentation in order to generate a response to the user. As the engineer in charge of maintaining this system, you want to evaluate the quality of the responses from your service.

Phoenix helps you:
- identify gaps in your documentation
- detect queries for which the LLM gave bad responses
- detect failures to retrieve relevant documents

In this tutorial, you will:

- Ask questions of a LangChain application backed by Qdrant over a knowledge base of the Arize documentation
- Use Phoenix to visualize user queries and knowledge base documents to identify areas of user interest not answered by your documentation
- Find clusters of responses with negative user feedback
- Identify failed retrievals using query density, cosine similarity, query distance, and LLM-assisted ranking metrics



## Chatbot Architecture

The architecture of your chatbot is shown below and can be explained in five steps.

1. The user sends a query about Arize to your service.
1. `langchain.embeddings.OpenAIEmbeddings` makes a request to OpenAI to embed the user query using the text-embedding-ada-002 model.
1. We retrieve by searching against the entries of your Qdrant database for the most similar pieces of context by MMR.
1. `langchain.llms.ChatOpenAI` generates a response by formatting the query and retrieved context into a single prompt and sending a request to OpenAI with the gpt-4-turbo-preview model.
1. The response is returned to the user.

Phoenix makes your search and retrieval system observable by capturing the inputs and outputs of these steps for analysis, including:

- your query embeddings
- the retrieved documents and similarity scores (relevance) to each query
- the generated response that is return to the user

With that overview in mind, let's dive into the notebook.

## 1. Install needed dependencies and import relevant packages

---

In [ ]:
!pip install --upgrade langchain qdrant-client langchain_community tiktoken cohere langchain-openai "protobuf>=3.20.3" "arize-phoenix[evals, llama-index]" "openai>=1"

Import libraries.

In [ ]:
# Standard library imports
import os
from getpass import getpass

# Third-party library imports
import nest_asyncio
import numpy as np
import openai
import pandas as pd

# Phoenix imports
import phoenix as px
from langchain.callbacks import StdOutCallbackHandler

# LangChain imports
from langchain.chains import RetrievalQA
from langchain.document_loaders import GitbookLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain_openai import ChatOpenAI
from phoenix.trace.langchain import LangChainInstrumentor

# Miscellaneous imports
from tqdm import tqdm

# Configuration and Initialization
nest_asyncio.apply()
pd.set_option("display.max_colwidth", None)

## 2. Configure Your OpenAI API Key

---

In [ ]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

## 3. Configure your Qdrant client in memory

We need to configure the embeddings to be used as well as the documents to be used. In this example, the documents come from arize's documentation

---

In [ ]:
model_name = "text-embedding-ada-002"

embeddings = OpenAIEmbeddings(model=model_name, openai_api_key=openai_api_key)

In [ ]:
def load_gitbook_docs(docs_url):
    """
    Loads documentation from a Gitbook URL.
    """

    loader = GitbookLoader(
        docs_url,
        load_all_paths=True,
    )
    return loader.load()


docs_url = "https://docs.arize.com/arize/"
docs = load_gitbook_docs(docs_url)

We build our qdrant vectorstore in memory for this example, however additional alternatives can be found in both Langchain's and Qdrant's documentation.

In [ ]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    location=":memory:",
    collection_name="my_documents",
)

## 4. Instrument LangChain

In [ ]:
LangChainInstrumentor().instrument()

## 5. Build Your LangChain Application

---

This example uses a `RetrievalQA` chain over a pre-built index of the Arize documentation, but you can use whatever LangChain application you like.

In [ ]:
handler = StdOutCallbackHandler()


num_retrieved_documents = 2
retriever = qdrant.as_retriever(
    search_type="mmr", search_kwargs={"k": num_retrieved_documents}, enable_limit=True
)
chain_type = "stuff"  # stuff, refine, map_reduce, and map_rerank
chat_model_name = "gpt-4-turbo-preview"
llm = ChatOpenAI(model_name=chat_model_name, temperature=0.0)
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type=chain_type,
    retriever=retriever,
    metadata={"application_type": "question_answering"},
    callbacks=[handler],
)

To use Phoenix, you must load your data into Pandas dataframes. First, load your knowledge base into a dataframe.


In [ ]:
database_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-phoenix-assets/datasets/unstructured/llm/context-retrieval/langchain-pinecone/database.parquet"
)

The columns of your dataframe are:

text: the chunked text in your knowledge base
text_vector: the embedding vector for the text, computed during the LangChain build using the "text-embedding-ada-002" embedding model from OpenAI

Next, download a dataframe containing query data.

In [ ]:
query_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-phoenix-assets/datasets/unstructured/llm/context-retrieval/langchain-pinecone/langchain_pinecone_query_dataframe_with_user_feedbackv2.parquet"
)

Define a schema to tell Phoenix what the columns of your query and database dataframes represent (features, predictions, actuals, tags, embeddings, etc.). See the docs for guides on how to define your own schema and API reference on phoenix.Schema and phoenix.EmbeddingColumnNames.

In [ ]:
query_schema = px.Schema(
    prompt_column_names=px.EmbeddingColumnNames(
        raw_data_column_name="text",
        vector_column_name="text_vector",
    ),
    response_column_names="response",
    tag_column_names=[
        "context_text_0",
        "context_similarity_0",
        "context_text_1",
        "context_similarity_1",
        "euclidean_distance_0",
        "euclidean_distance_1",
        "openai_relevance_0",
        "openai_relevance_1",
        "openai_precision@1",
        "openai_precision@2",
        "user_feedback",
    ],
)
database_schema = px.Schema(
    document_column_names=px.EmbeddingColumnNames(
        raw_data_column_name="text",
        vector_column_name="text_vector",
    ),
)

Create Phoenix datasets that wrap your dataframes with the schemas that describe them.

In [ ]:
database_ds = px.Dataset(
    dataframe=database_df,
    schema=database_schema,
    name="qdrant",
)
query_ds = px.Dataset(
    dataframe=query_df,
    schema=query_schema,
    name="query",
)

Launch Phoenix. Note that the database dataset is passed in as the corpus - a reserved keyword argument for passing in the knowledge base for search and retrieval analysis. Follow the instructions in the cell output to open the Phoenix UI.

In [ ]:
session = px.launch_app(query_ds, corpus=database_ds)

In order to make your LLM application observable, it must be instrumented. That is, the code must emit traces. The instrumented data must then be sent to an Observability backend, in our case the Phoenix server.

The columns of the dataframe are:

text: the query text
text_vector: the embedding representation of the query, captured from LangChain at query time
response: the final response from the LangChain application
context_text_0: the first retrieved context from the knowledge base
context_similarity_0: the cosine similarity between the query and the first retrieved context
context_text_1: the second retrieved context from the knowledge base
context_similarity_1: the cosine similarity between the query and the first retrieved context
user_feedback: approval or rejection from the user (-1 means thumbs down, +1 means thumbs up)

Lets try out running out the first 10 queries on the query_df by using Qdrant as retriever!

In [ ]:
for i in range(10):
    row = query_df.iloc[i]
    response = chain.invoke(row["text"])
    print(response)

The query and database datasets are drawn from different distributions; the queries are short questions while the database entries are several sentences to a paragraph. The embeddings from OpenAI's "text-embedding-ada-002" capture these differences and naturally separate the query and context embeddings into distinct regions of the embedding space. When using Phoenix, you want to "overlay" the query and context embedding distributions so that queries appear close to their retrieved context in the Phoenix point cloud. To achieve this, we compute a centroid for each dataset that represents an average point in the embedding distribution and center the two distributions so they overlap.

In [ ]:
database_centroid = database_df["text_vector"].mean()
database_df["text_vector"] = database_df["text_vector"].apply(
    lambda x: x - database_centroid
)
query_centroid = query_df["text_vector"].mean()
query_df["text_vector"] = query_df["text_vector"].apply(lambda x: x - query_centroid)

## 6. Run LLM assisted Evals

Cosine similarity and Euclidean distance are reasonable proxies for retrieval quality, but they don't always work perfectly. A novel idea is to use LLMs to measure retrieval quality by simply asking the LLM whether each piece of retrieved context is relevant or irrelevant to the corresponding query.

⚠️ It's strongly recommended to use GPT-4 for this step if you have access, since we've found it to be more trustworthy for this particular task.

💬 Use OpenAI to predict whether each retrieved document is relevant or irrelevant to the query.

In [ ]:
from phoenix.evals import OpenAIModel, llm_classify, RAG_RELEVANCY_PROMPT_TEMPLATE, RAG_RELEVANCY_PROMPT_RAILS_MAP

In [ ]:
# construct an eval dataset with the "input" and "reference" columns to pass into the classifier
eval_dataset = query_df.copy()
eval_dataset["input"] = eval_dataset["text"]
eval_dataset["reference"] = eval_dataset["context_text_0"] + "/n" + eval_dataset["context_text_1"]

model = OpenAIModel(model="gpt-4")
relevance_classifications = llm_classify(eval_dataset, model=model, template=RAG_RELEVANCY_PROMPT_TEMPLATE, rails=list(RAG_RELEVANCY_PROMPT_RAILS_MAP.values()), provide_explanation=True)
relevance_classifications.head()

In [ ]:
print(f"🚀 Open the Phoenix UI if you haven't already: {session.url}")